In [20]:
import pandas as pd
from mysql import connector

In [21]:
# Step 1: Connect to MySQL server
connection = connector.connect(
    host="localhost",
    user="root",
    port=3306,
    password="123456789",
    connection_timeout=600,  # Timeout after 600 seconds
)

mycursor = connection.cursor()


In [22]:
# Step 2: Create database and select it
mycursor.execute("CREATE DATABASE IF NOT EXISTS BusDetails")
mycursor.execute("USE BusDetails")

In [23]:
# Step 3: Define the table schema
table_name = "bus_routes"
column_definitions = """
id INT AUTO_INCREMENT PRIMARY KEY,
route_name TEXT,
route_link TEXT,
busname TEXT,
bustype TEXT,
departing_time TIME,
duration TEXT,
reaching_time TIME,
star_rating FLOAT,
price DECIMAL(10, 2),
seats_available INT
"""

# Create table
mycursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({column_definitions})")
print(f"Table {table_name} created successfully.")

Table bus_routes created successfully.


In [24]:
# Step 4: Read the CSV file
file_path = r"C:\Users\ashwi\GUVI_Projects\Redbus_Data_Scrapping\bus_details.csv"
try:
    data = pd.read_csv(file_path)
    data = data.dropna()  # Drop rows with null values
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error reading the CSV file: {e}")
    connection.close()
    exit()
data['Seat_Availability'] = data['Seat_Availability'].str.extract(r'(\d+)')

CSV file loaded successfully.
           Route_Name                                         Route_Link  \
0  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   
1  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   
2  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   
3  Guwahati to Tezpur  https://www.redbus.in/bus-tickets/guwahati-to-...   
4  Guwahati to Tezpur  https://www.redbus.in/bus-tickets/guwahati-to-...   

                                            Bus_Name  \
0  Assam State Transport Corporation (ASTC) - 139847   
1                    BLUE HILL TRAVELS INDIA LIMITED   
2                    BLUE HILL TRAVELS INDIA LIMITED   
3                                  WARISPIYA TRAVELS   
4                                    Kalpana Travels   

                            Bus_Type Departing_Time Duration Reaching_Time  \
0                Volvo AC Seater 2+2          16:15  04h 15m         20:30   
1  NON A/C Seater Se

In [25]:
# Step 5: Insert data into the table
try:
    for i in range(len(data)):
        # Convert row to tuple and handle data type conversion for floats
        row = tuple(
            [float(val) if isinstance(val, float) else val for val in data.iloc[i]]
        )
        placeholders = ", ".join(["%s"] * len(row))  # Generate placeholders
        query = f"INSERT INTO {table_name} (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) VALUES ({placeholders})"
        mycursor.execute(query, row)  # Parameterized query to insert data

    connection.commit()
    print("Data inserted successfully.")
except Exception as e:
    print(f"Error inserting data into the table: {e}")
    connection.close()
    exit()


Data inserted successfully.


In [26]:
# Step 6: Retrieve and display data
try:
    print("\nTables in the database:")
    mycursor.execute("SHOW TABLES")
    for db in mycursor:
        print(db)

    print("\nData in the table:")
    mycursor.execute(f"SELECT * FROM {table_name}")
    for record in mycursor:
        print(record)
except Exception as e:
    print(f"Error retrieving data: {e}")
finally:
    # Step 7: Close the connection
    connection.close()
    print("Connection closed.")


Tables in the database:
('bus_routes',)

Data in the table:
(1, 'Tezpur to Guwahati', 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'Assam State Transport Corporation (ASTC) - 139847', 'Volvo AC Seater 2+2', datetime.timedelta(seconds=58500), '04h 15m', datetime.timedelta(seconds=73800), 4.1, Decimal('298.00'), 5)
(2, 'Tezpur to Guwahati', 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'BLUE HILL TRAVELS INDIA LIMITED', 'NON A/C Seater Semi Sleeper (2+1)', datetime.timedelta(seconds=76500), '05h 45m', datetime.timedelta(seconds=10800), 3.4, Decimal('450.00'), 19)
(3, 'Tezpur to Guwahati', 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'BLUE HILL TRAVELS INDIA LIMITED', 'NON A/C Seater Semi Sleeper (2+1)', datetime.timedelta(seconds=78300), '05h 30m', datetime.timedelta(seconds=11700), 3.6, Decimal('450.00'), 14)
(4, 'Guwahati to Tezpur', 'https://www.redbus.in/bus-tickets/guwahati-to-tezpur', 'WARISPIYA TRAVELS', 'Bharat Benz A/C Seater (2+2)', datetime.tim

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



(35600, 'Kurnool to Hyderabad', 'https://www.redbus.in/bus-tickets/kurnool-to-hyderabad', 'APSRTC - 6411', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=82800), '04h 15m', datetime.timedelta(seconds=11700), 2.5, Decimal('377.00'), 27)
(35601, 'Kurnool to Hyderabad', 'https://www.redbus.in/bus-tickets/kurnool-to-hyderabad', 'APSRTC - 51529', 'STAR LINER(NON-AC SLEEPER 2+1)', datetime.timedelta(seconds=83160), '04h 09m', datetime.timedelta(seconds=11700), 4.3, Decimal('488.00'), 9)
(35602, 'Kurnool to Hyderabad', 'https://www.redbus.in/bus-tickets/kurnool-to-hyderabad', 'APSRTC - 6513', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=83700), '04h 05m', datetime.timedelta(seconds=12000), 4.0, Decimal('377.00'), 30)
(35603, 'Kurnool to Hyderabad', 'https://www.redbus.in/bus-tickets/kurnool-to-hyderabad', 'APSRTC - 6600', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=84000), '04h 45m', datetime.timedelta(seconds=14700), 3.7,